In [ ]:
import tensorflow as tf
import numpy as np
import tkinter as tk
from tkinter import Canvas, Button, Label, Frame
from PIL import Image, ImageDraw, ImageOps

# 1. Càrrega i preparació del conjunt de dades MNIST
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalitzar les imatges a valors entre 0 i 1
train_images = train_images / 255.0
test_images = test_images / 255.0

# 2. Construir el model de xarxa neuronal convolucional (CNN)
model = tf.keras.Sequential([

    tf.keras.layers.Reshape((28, 28, 1), input_shape=(28, 28)),  # Capa de reestructuració per adaptar l'entrada a les dimensions esperades per les capes convolucionals
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),     # Primera capa convolucional amb 32 filtres de mida 3x3 i funció d'activació ReLU
    tf.keras.layers.MaxPooling2D((2, 2)),     # Capa de max pooling per reduir les dimensions espacials
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),     # Segona capa convolucional amb 64 filtres de mida 3x3 i funció d'activació ReLU
    tf.keras.layers.MaxPooling2D((2, 2)),     # Segona capa de max pooling per reduir encara més les dimensions espacials
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),     # Tercera capa convolucional amb 64 filtres de mida 3x3 i funció d'activació ReLU
    tf.keras.layers.Flatten(),     # Aplanar les característiques extretes per alimentar-les a les capes densament connectades
    tf.keras.layers.Dense(64, activation='relu'),     # Capa densa amb 64 neurones i funció d'activació ReLU
    tf.keras.layers.Dense(10, activation='softmax')     # Capa de sortida amb 10 neurones (una per cada dígit) i funció d'activació softmax

])

# Compilació del model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrenament del model
model.fit(train_images, train_labels, epochs=5, validation_split=0.2)

# 3. Crear la interfície gràfica
class DrawingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Reconixement de Dígits")
        self.root.geometry("400x500")
        self.root.configure(bg='lightblue')

        # Crear un marc per al canvas i els botons
        self.frame = Frame(root, bg='lightgray', padx=10, pady=10)
        self.frame.pack(pady=10)

        # Canvas per dibuixar números
        self.canvas = Canvas(self.frame, width=280, height=280, bg='white', relief='raised', borderwidth=2)
        self.canvas.pack(pady=10)

        # Marc per als botons
        self.button_frame = Frame(root, bg='lightblue')
        self.button_frame.pack()

        # Botó per predir el dígit
        self.predict_button = Button(self.button_frame, text="Predir", command=self.predict, width=15, state='disabled')
        self.predict_button.pack(side='left', padx=5)

        # Botó per esborrar el canvas
        self.clear_button = Button(self.button_frame, text="Esborrar", command=self.clear_canvas, width=15)
        self.clear_button.pack(side='left', padx=5)

        # Etiqueta per mostrar el resultat de la predicció
        self.result_label = Label(root, text="Predicció: ", font=('Helvetica', 16), bg='lightblue')
        self.result_label.pack(pady=10)

        # Crear una imatge en blanc per dibuixar-hi sobre
        self.image = Image.new("L", (280, 280), 255)
        self.draw = ImageDraw.Draw(self.image)

        # Vincular l'esdeveniment de dibuix al canvas
        self.canvas.bind("<B1-Motion>", self.paint)

    def paint(self, event):
        # Funció per dibuixar en el canvas i en la imatge
        x, y = event.x, event.y
        self.canvas.create_oval(x, y, x+10, y+10, fill='black', outline='black')  # Gruix del llapis de 10 píxels
        self.draw.ellipse([x, y, x+10, y+10], fill='black')  # Gruix del llapis de 10 píxels
        self.predict_button.config(state='normal')

    def predict(self):
        # Funció per realitzar la predicció del dígit dibuixat
        if np.array(self.image).sum() == 255 * 280 * 280:
            self.result_label.config(text="No hi ha res dibuixat.")
            return

        # Preparar la imatge per a la predicció
        image_resized = self.image.resize((28, 28), Image.LANCZOS)
        image_inverted = ImageOps.invert(image_resized)
        image_array = np.array(image_inverted) / 255.0
        image_array = image_array.reshape((1, 28, 28, 1))

        # Realitzar la predicció amb el model
        predictions = model.predict(image_array)
        predicted_label = np.argmax(predictions)
        predicted_confidence = np.max(predictions)

        # Mostrar el resultat de la predicció
        self.result_label.config(text=f"Predicció: {predicted_label} (Confiança: {predicted_confidence:.2f})")

    def clear_canvas(self):
        # Funció per netejar el canvas i reiniciar la imatge
        self.canvas.delete("all")
        self.image = Image.new("L", (280, 280), 255)
        self.draw = ImageDraw.Draw(self.image)
        self.result_label.config(text="Predicció: ")
        self.predict_button.config(state='disabled')

# Executar l'aplicació
root = tk.Tk()
app = DrawingApp(root)
root.mainloop()
